# This notebook is meant to be a demo of what can be done with intake on ref-marc datas

In [3]:
import intake
import dask_hpcconfig
from distributed import Client
import xarray as xr
import hvplot.xarray
import geoviews.feature as gf
import os
catalogue="/home/datawork-taos-s/intranet/kerchunk/ref-marc.yaml"

Detail of the imports:
-----------------------------------------------------------------------------------------------  
    Intake    
>- Handle the catalog contained in the yaml files.

    dask_hpcconfig    
>- Parrallelize your tasks on an hpc like datatrmor.

    Distributed import Client    
>- Client is creating the dask client

    Xarray    
>- Xarray makes operation on datasets and creates labbeled multidimensional arrays

    hvplot.xarray
>- Plotting 

    geoviews.features
>- Plotting that enables some features on maps 

    os
>- Can handle operating system dependant functionnality 

In [2]:
cluster = dask_hpcconfig.cluster("datarmor-local")
cluster.scale(14)
client = Client(cluster)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /user/todaka/proxy/8787/status,
Dashboard: /user/todaka/proxy/8787/status,Workers: 14
Total threads: 98,Total memory: 201.25 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:45965,Workers: 14
Dashboard: /user/todaka/proxy/8787/status,Total threads: 98
Started: Just now,Total memory: 201.25 GiB
Comm: tcp://127.0.0.1:34276,Total threads: 7
Dashboard: /user/todaka/proxy/35598/status,Memory: 14.38 GiB
Nanny: tcp://127.0.0.1:34308,


To vizaulise the dask-lab extension you need to go the the dask-lab tab and fill it wih :
>- /user/{ JUPYTERHUB-USERNAME }/proxy/{ PORT-NUMBER }

In [29]:
def list_param(cat, param):
    return cat.metadata["parameters"][param]["allowed"]

def allowed_param(cat):
    return cat.metadata["parameters"]

def allowed_years(data):
#    path = os.path.dirname(catalogue) + data
    path = catalogue.replace('.yaml','/') + data
    files = [f.replace('.json.zst','') for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]
#    print(files)
    files.sort()
    return files

def year_concat(cat, source, region):
    concat = [cat[source](region=region, year=i).to_dask() 
              for i in allowed_years(region)]

    return xr.concat(
        concat, 
        dim="time", 
        coords="minimal", 
        data_vars="minimal", 
        compat="override"
    )

Detail of the functions:
-----------------------------------------------------------------------------------------------  
    list_param(cat, param)    
>- Takes 2 arguments: cat = An opened catalog, param = The name of a parameter.
>- Returns a ptyhon list containing all the parameters.

    allowed_param(cat)    
>- Takes 1 argument: cat = An opened catalog
>- Returns a python dict containing the allowed values of each parameters and their description

    Allowed_years(data)    
>- Takes 1 argument: data = A region of data.
>- Returns a pythton list containing all the files stored in the directory. The combined jsons are stored in a directory named after the region (Same name than in the ref-marc directory)  
/!\ In this notebook, it will only work with ref-marc data because the json files are located at /home/datawork-lops-iaocea/catalog/kerchunk/ref-marc/. To use it to another directory, update the variable "path" in the function

    year_concat(cat, source, region)    
>- Takes 3 arguments: cat = An opened catalog, source = the source from this catalog, region = the region where the data comes from
>- Returns: A dataset concatenated over the time dimension


In [12]:
cat = intake.open_catalog(catalogue)

#### To open the catalog you can use the cell above, you just need to give the path to an intake catalog.

In [13]:
regions = list_param(
    cat, "region"
)

In [14]:
regions

['f1_e2500_agrif/MARC_F1-MARS3D-ADOUR',
 'f1_e2500_agrif/MARC_F1-MARS3D-ARMOR',
 'f1_e2500_agrif/MARC_F1-MARS3D-FINIS',
 'f1_e2500_agrif/MARC_F1-MARS3D-GIRONDE',
 'f1_e2500_agrif/MARC_F1-MARS3D-LOIRE',
 'f1_e2500_agrif/MARC_F1-MARS3D-MANGAE2500-AGRIF',
 'f1_e2500_agrif/MARC_F1-MARS3D-MORBIHAN',
 'f1_e2500_agrif/MARC_F1-MARS3D-PDC',
 'f1_e2500_agrif/MARC_F1-MARS3D-SEINE',
 'l1_v9/MARC_L1-MARS2D-AQUI250',
 'l1_v9/MARC_L1-MARS2D-ATLNE2000',
 'l1_v9/MARC_L1-MARS2D-FINIS250',
 'l1_v9/MARC_L1-MARS2D-MANE250',
 'l1_v9/MARC_L1-MARS2D-MANGA700',
 'l1_v9/MARC_L1-MARS2D-MANW250',
 'l1_v9/MARC_L1-MARS2D-SUDBZH250',
 'b1',
 'b1_v10',
 'b1_v10r2019',
 'f1_e2500',
 'f1_e4000',
 'f2_1200',
 'f2_1200_sn',
 'f2_1200_v10',
 'l1_brest',
 'l2_v10',
 'ww3/ANTILLES-3MIN',
 'ww3/ANTILLES-UG',
 'ww3/ATNE-10M',
 'ww3/FINIS-200M',
 'ww3/GLOBAL-30MIN',
 'ww3/MED-6MIN',
 'ww3/MENOR-2MIN',
 'ww3/MENOR-UG',
 'ww3/NORGAS-2MIN',
 'ww3/NORGAS-UG',
 'ww3/NORGAS-UG_V3',
 'ww3/REUNION-UG']

### This tab is here to test the file and see how long it takes to be concatenated, if everything works as it should and measure the performance

Tested on datarmor with the spark notebook







|    REGION                               |  OPEN DATASET  |      CONCAT     |   COMPUTE   |     PLOT      |Concat time (s) |Nbr of Years |
|    :------------------------------------|:--------------:|:---------------:|:-----------:|:-------------:|:--------------:|:-----------:|
|   ANTILLES-UG                           |Works           |Missing variable |             |               |                |             |
|   ATNE-10M                              |Works           |Missing variable |             |               |                |             |
|   CALEDONIE-3MIN                        |Works           |Missing variable |             |               |                |             |
|   GLOBAL-30MIN                          |Works           |Missing variable |             |               |                |             |
|   MARC_F1-MARS3D-ADOUR                  |Works           |Works            |Works        |Works          |23.6            |9            |
|   MARC_F1-MARS3D-ARMOR                  |Works           |Works            |Works        |Works          |23              |9            |
|   MARC_F1-MARS3D-FINIS                  |Works           |Works            |Works        |Works          |20.9            |9            |
|   MARC_F1-MARS3D-GIRONDE                |Works           |Works            |Works        |Works          |21.1            |9            |
|   MARC_F1-MARS3D-LOIRE                  |Works           |Works            |Works        |Works          |20              |9            |
|   MARC_F1-MARS3D-MANGAE2500-AGRIF       |Works           |Works            |Works        |Works          |563 (ms)        |2            |
|   MARC_F1-MARS3D-MORBIHAN               |Works           |Works            |Works        |Works          |17              |6            |
|   MARC_F1-MARS3D-PDC                    |Works           |Works            |Works        |Works          |117 (1.57 s)    |9            |
|   MARC_F1-MARS3D-SEINE                  |Works           |Works            |Works        |Works          |29.8            |9            |
|   MED-6MIN                              |Works           |Missing variable |             |               |                |12           |
|   MENOR-2MIN                            |Works           |Missing variable |             |               |                |12           |
|   MENOR-UG                              |Works           |Diff  dims size  |             |               |                |12           |
|   NORGAS-2MIN                           |Works           |Missing variable |             |               |                |12           |
|   NORGAS-UG                             |Works           |Missing variable |             |               |                |15           |
|   NORGAS-UG_V3                          |Works           |Works            |Works        |               |20.5            |12           |
|   POLYNESIE-3MIN                        |Works           |Missing variable |             |               |                |             |
|   REUNION-180M                          |Works           |Works            |Works        |Works          |453 (ms)        |2            |
|   REUNION-UG                            |Works           |Missing variable |             |               |                |12           |
|   b1                                    |Works           |Works            |Works        |fails          |1.95            |5            |
|   b1_v10                                |Works           |Missing variable |             |               |                |11           |
|   b1_v10r2019                           |Works           |Works            |Works        |Works          |2.43            |11           |
|   f1_e2500                              |Works           |Works            |Works        |Works          |127 (2m07)      |11           |
|   f1_e4000                              |Works           |Works            |Works        |Works          |128 (2m08)      |17           |
|   f2_1200                               |Works           |Works            |Works        |Works          |23.4            |10           |
|   f2_1200_sn                            |Works           |Works            |Works        |Works          |2.84            |4            |
|   f2_1200_v10                           |Works           |Works            |Works        |Works          |18.2            |8            |
|   l2_v10                                |Works           |Works            |Works        |Works          |8.17            |9            |
|                                         |                |                 |             |               |                |             |


In [15]:
region = 'f1_e2500_agrif/MARC_F1-MARS3D-SEINE'

In [20]:
filename = allowed_years(data=region)
#cat.marc(region=region,year="2021.json.zst").to_dask()
cat.marc(region=region,year='2018').to_dask()

<xarray.Dataset>
Dimensions:       (level: 40, nj: 183, ni: 413, nj_u: 183, ni_u: 413,
                   nj_v: 183, ni_v: 413, time: 8760, ni_f: 413, nj_f: 183)
Coordinates: (12/16)
    latitude      (nj, ni) float64 dask.array<chunksize=(183, 413), meta=np.ndarray>
    latitude_u    (nj_u, ni_u) float64 dask.array<chunksize=(183, 413), meta=np.ndarray>
    latitude_v    (nj_v, ni_v) float64 dask.array<chunksize=(183, 413), meta=np.ndarray>
  * level         (level) float32 -0.9875 -0.9625 -0.9375 ... -0.0375 -0.0125
    longitude     (nj, ni) float64 dask.array<chunksize=(183, 413), meta=np.ndarray>
    longitude_u   (nj_u, ni_u) float64 dask.array<chunksize=(183, 413), meta=np.ndarray>
    ...            ...
  * ni_v          (ni_v) float32 0.0 1.0 2.0 3.0 4.0 ... 409.0 410.0 411.0 412.0
  * nj            (nj) float32 0.0 1.0 2.0 3.0 4.0 ... 179.0 180.0 181.0 182.0
  * nj_f          (nj_f) float32 0.5 1.5 2.5 3.5 4.5 ... 179.5 180.5 181.5 182.5
  * nj_u          (nj_u) float32 0.0 1.0 2.0 3.0 4.0 ... 179.0 180.0 181.0 182.0
  * nj_v          (nj_v) float32 0.5 1.5 2.5 3.5 4.5 ... 179.5 180.5 181.5 182.5
  * time          (time) datetime64[us] 2018-01-01 ... 2018-12-31T23:00:00
Data variables: (12/18)
    Csu_sig       (level) float32 dask.array<chunksize=(40,), meta=np.ndarray>
    H0            (nj, ni) float32 dask.array<chunksize=(183, 413), meta=np.ndarray>
    HX            (nj_u, ni_u) float32 dask.array<chunksize=(183, 413), meta=np.ndarray>
    HY            (nj_v, ni_v) float32 dask.array<chunksize=(183, 413), meta=np.ndarray>
    SAL           (time, level, nj, ni) float32 dask.array<chunksize=(1, 40, 183, 413), meta=np.ndarray>
    SIG           (level) float32 dask.array<chunksize=(40,), meta=np.ndarray>
    ...            ...
    adapt_imp_ts  (time, level, nj, ni) float32 dask.array<chunksize=(1, 40, 183, 413), meta=np.ndarray>
    adapt_imp_uz  (time, level, nj, ni) float32 dask.array<chunksize=(1, 40, 183, 413), meta=np.ndarray>
    adapt_imp_vz  (time, level, nj, ni) float32 dask.array<chunksize=(1, 40, 183, 413), meta=np.ndarray>
    b             float32 ...
    hc            (nj, ni) float32 dask.array<chunksize=(183, 413), meta=np.ndarray>
    theta         float32 ...
Attributes: (12/52)
    Conventions:             CF-1.6 OCO-1.3.1 COMODO-1.0
    arakawa_grid_type:       C1
    area:                    SEINE
    comment:                 Use of Meteo-France AROME meteorological data
    contact:                 cdoco-exploit@ifremer.fr
    creation_date:           2019-07-26T09:26:18Z
    ...                      ...
    run_time:                2019-07-26T09:26:18Z
    software_version:        MARC forecasting system v2
    source:                  MARS MANGAE2500-AGRIF
    southernmost_latitude:   49.2424
    title:                   MARC F1 MANGAE2500 AGRIF hindcast
    westernmost_longitude:   -1.4598

In [30]:
allowed_years(region)

['2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018']

In [31]:
%%time
ds = year_concat(
    cat, "marc", region
)

CPU times: user 16.9 s, sys: 984 ms, total: 17.9 s
Wall time: 18.5 s


In [40]:
%%time
da=ds.XE.unify_chunks()


CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 984 µs


In [ ]:
da.hvplot.quadmesh(
     x="longitude", y="latitude", rasterize=True, geo=True, cmap="BuPu",coastline=True)
#) * gf.land.opts(scale="10m") * gf.coastline.opts(scale="10m")